# T81-558: Applications of Deep Neural Networks
**Module 12: Deep Learning and Security**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module Video Material

Main video lecture:

* Part 12.1: Introduction to the OpenAI Gym [[Video]](https://www.youtube.com/playlist?list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_12_reinforcement.ipynb)
* Part 12.2: Introduction to Q-Learning for Keras [[Video]](https://www.youtube.com/playlist?list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_12_reinforcement.ipynb)
* **Part 12.3: Keras Q-Learning in the OpenAI Gym** [[Video]](https://www.youtube.com/playlist?list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_12_reinforcement.ipynb)
* Part 12.4: Atari Games with Keras Neural Networks [[Video]](https://www.youtube.com/playlist?list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_12_reinforcement.ipynb)
* 12.5: How Alpha Zero used Reinforcement Learning to Master Chess [[Video]](https://www.youtube.com/playlist?list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_12_reinforcement.ipynb)


# Part 12.3: Keras Q-Learning in the OpenAI Gym

![Deep Q-Learning](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/deepqlearning.png "Reinforcement Learning")

* **CEMAgent**
    * **model** - The neural network that will be trained.
    * **nb_actions** - The number of actions the agent can take (e.g. up, down, left, right, fire)
    * **memory** - The EpisodeParameterMemory object to use.  This object observes and save all of the state transitions so that you can train your network on them later on (instead of having to make observations from the environment all the time).
    * **batch_size** - The batch size for neural network training, same concept as deep learning batch sizes.
    * **nb_steps_warmup** - Number of training steps to pass before any learning occurs.
    * **train_interval** - Logging interval, defines how often to log.
    * **elite_frac**
* **CEMAgent.fit**
    * **env** - The OpenAI gym environment being used.
    * **nb_steps** - Number of training steps to be performed.
    * **visualize** - If `True`, the environment is visualized during training. However,
                this is likely going to slow down training significantly and is thus intended to be
                a debugging instrument.
    * **verbose** - 0 for no logging, 1 for interval logging (compare `log_interval`), 2 for episode logging




In [2]:
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents.cem import CEMAgent
from rl.memory import EpisodeParameterMemory

ENV_NAME = 'CartPole-v0'


# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)

nb_actions = env.action_space.n
obs_dim = env.observation_space.shape[0]

# Option 1 : Simple model
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(nb_actions))
model.add(Activation('softmax'))

# Option 2: deep network
# model = Sequential()
# model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(nb_actions))
# model.add(Activation('softmax'))


print(model.summary())


# Finally, we configure and compile our agent. You can use every built-in tensorflow.keras optimizer and
# even the metrics!
memory = EpisodeParameterMemory(limit=1000, window_length=1)

cem = CEMAgent(model=model, nb_actions=nb_actions, memory=memory,
               batch_size=50, nb_steps_warmup=2000, train_interval=50, elite_frac=0.05)
cem.compile()

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
cem.fit(env, nb_steps=100000, visualize=False, verbose=2)

# After training is done, we save the best weights.
cem.save_weights('cem_{}_params.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
cem.test(env, nb_episodes=5, visualize=True)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 10        
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
None
Training for 100000 steps ...
    47/100000: episode: 1, duration: 0.045s, episode steps:  47, steps per second: 1050, episode reward: 47.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.426 [0.000, 1.000],  mean_best_reward: --
    90/100000: episode: 2, duration: 0.020s, episode steps:  43, steps per second: 2115, episode reward: 43.000, mean reward:  1.000 [ 1.000,  1.

  1148/100000: episode: 50, duration: 0.017s, episode steps:  33, steps per second: 1971, episode reward: 33.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.485 [0.000, 1.000],  mean_best_reward: --
  1160/100000: episode: 51, duration: 0.007s, episode steps:  12, steps per second: 1808, episode reward: 12.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.667 [0.000, 1.000],  mean_best_reward: --
  1181/100000: episode: 52, duration: 0.011s, episode steps:  21, steps per second: 1937, episode reward: 21.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.333 [0.000, 1.000],  mean_best_reward: --
  1196/100000: episode: 53, duration: 0.009s, episode steps:  15, steps per second: 1644, episode reward: 15.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.600 [0.000, 1.000],  mean_best_reward: --
  1206/100000: episode: 54, duration: 0.006s, episode steps:  10, steps per second: 1726, episode reward: 10.000, mean reward:  1.000 [ 1.000,  1.000], mean action:

  1946/100000: episode: 90, duration: 0.007s, episode steps:  12, steps per second: 1635, episode reward: 12.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.167 [0.000, 1.000],  mean_best_reward: --
  1959/100000: episode: 91, duration: 0.007s, episode steps:  13, steps per second: 1830, episode reward: 13.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.769 [0.000, 1.000],  mean_best_reward: --
  1979/100000: episode: 92, duration: 0.012s, episode steps:  20, steps per second: 1739, episode reward: 20.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.350 [0.000, 1.000],  mean_best_reward: --
  1989/100000: episode: 93, duration: 0.007s, episode steps:  10, steps per second: 1421, episode reward: 10.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.000 [0.000, 0.000],  mean_best_reward: --
  2023/100000: episode: 94, duration: 0.017s, episode steps:  34, steps per second: 1964, episode reward: 34.000, mean reward:  1.000 [ 1.000,  1.000], mean action:

  3229/100000: episode: 136, duration: 0.035s, episode steps:  74, steps per second: 2141, episode reward: 74.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
  3263/100000: episode: 137, duration: 0.019s, episode steps:  34, steps per second: 1768, episode reward: 34.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
  3300/100000: episode: 138, duration: 0.018s, episode steps:  37, steps per second: 2090, episode reward: 37.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.486 [0.000, 1.000],  mean_best_reward: --
  3338/100000: episode: 139, duration: 0.017s, episode steps:  38, steps per second: 2194, episode reward: 38.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
  3375/100000: episode: 140, duration: 0.018s, episode steps:  37, steps per second: 2036, episode reward: 37.000, mean reward:  1.000 [ 1.000,  1.000], mean ac

  5021/100000: episode: 186, duration: 0.027s, episode steps:  55, steps per second: 2018, episode reward: 55.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.509 [0.000, 1.000],  mean_best_reward: --
  5132/100000: episode: 187, duration: 0.049s, episode steps: 111, steps per second: 2250, episode reward: 111.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.495 [0.000, 1.000],  mean_best_reward: --
  5144/100000: episode: 188, duration: 0.006s, episode steps:  12, steps per second: 1926, episode reward: 12.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.667 [0.000, 1.000],  mean_best_reward: --
  5192/100000: episode: 189, duration: 0.022s, episode steps:  48, steps per second: 2218, episode reward: 48.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
  5248/100000: episode: 190, duration: 0.025s, episode steps:  56, steps per second: 2267, episode reward: 56.000, mean reward:  1.000 [ 1.000,  1.000], mean a

...

 92340/100000: episode: 2477, duration: 0.009s, episode steps:  15, steps per second: 1714, episode reward: 15.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.533 [0.000, 1.000],  mean_best_reward: --
 92368/100000: episode: 2478, duration: 0.013s, episode steps:  28, steps per second: 2134, episode reward: 28.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.429 [0.000, 1.000],  mean_best_reward: --
 92384/100000: episode: 2479, duration: 0.008s, episode steps:  16, steps per second: 2001, episode reward: 16.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.562 [0.000, 1.000],  mean_best_reward: --
 92415/100000: episode: 2480, duration: 0.014s, episode steps:  31, steps per second: 2149, episode reward: 31.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.484 [0.000, 1.000],  mean_best_reward: --
 92439/100000: episode: 2481, duration: 0.012s, episode steps:  24, steps per second: 1987, episode reward: 24.000, mean reward:  1.000 [ 1.000,  1.000], me

 94163/100000: episode: 2516, duration: 0.034s, episode steps:  71, steps per second: 2115, episode reward: 71.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.563 [0.000, 1.000],  mean_best_reward: --
 94294/100000: episode: 2517, duration: 0.059s, episode steps: 131, steps per second: 2237, episode reward: 131.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.519 [0.000, 1.000],  mean_best_reward: --
 94334/100000: episode: 2518, duration: 0.018s, episode steps:  40, steps per second: 2188, episode reward: 40.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
 94418/100000: episode: 2519, duration: 0.040s, episode steps:  84, steps per second: 2117, episode reward: 84.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.476 [0.000, 1.000],  mean_best_reward: --
 94453/100000: episode: 2520, duration: 0.017s, episode steps:  35, steps per second: 2008, episode reward: 35.000, mean reward:  1.000 [ 1.000,  1.000], m

 96096/100000: episode: 2563, duration: 0.031s, episode steps:  63, steps per second: 2052, episode reward: 63.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.524 [0.000, 1.000],  mean_best_reward: --
 96115/100000: episode: 2564, duration: 0.009s, episode steps:  19, steps per second: 2051, episode reward: 19.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.526 [0.000, 1.000],  mean_best_reward: --
 96137/100000: episode: 2565, duration: 0.011s, episode steps:  22, steps per second: 2003, episode reward: 22.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.591 [0.000, 1.000],  mean_best_reward: --
 96169/100000: episode: 2566, duration: 0.015s, episode steps:  32, steps per second: 2171, episode reward: 32.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.562 [0.000, 1.000],  mean_best_reward: --
 96212/100000: episode: 2567, duration: 0.021s, episode steps:  43, steps per second: 2096, episode reward: 43.000, mean reward:  1.000 [ 1.000,  1.000], me

 97750/100000: episode: 2608, duration: 0.029s, episode steps:  44, steps per second: 1510, episode reward: 44.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.545 [0.000, 1.000],  mean_best_reward: --
 97781/100000: episode: 2609, duration: 0.018s, episode steps:  31, steps per second: 1768, episode reward: 31.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.484 [0.000, 1.000],  mean_best_reward: --
 97896/100000: episode: 2610, duration: 0.088s, episode steps: 115, steps per second: 1314, episode reward: 115.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.513 [0.000, 1.000],  mean_best_reward: --
 97924/100000: episode: 2611, duration: 0.018s, episode steps:  28, steps per second: 1562, episode reward: 28.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.464 [0.000, 1.000],  mean_best_reward: --
 97941/100000: episode: 2612, duration: 0.014s, episode steps:  17, steps per second: 1227, episode reward: 17.000, mean reward:  1.000 [ 1.000,  1.000], m

 99339/100000: episode: 2650, duration: 0.053s, episode steps: 102, steps per second: 1921, episode reward: 102.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.520 [0.000, 1.000],  mean_best_reward: --
 99430/100000: episode: 2651, duration: 0.041s, episode steps:  91, steps per second: 2208, episode reward: 91.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.516 [0.000, 1.000],  mean_best_reward: 108.500000
 99458/100000: episode: 2652, duration: 0.013s, episode steps:  28, steps per second: 2121, episode reward: 28.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.536 [0.000, 1.000],  mean_best_reward: --
 99508/100000: episode: 2653, duration: 0.023s, episode steps:  50, steps per second: 2205, episode reward: 50.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
 99570/100000: episode: 2654, duration: 0.028s, episode steps:  62, steps per second: 2206, episode reward: 62.000, mean reward:  1.000 [ 1.000,  1